In [1]:
from qgis.core import QgsApplication, QgsRasterLayer, QgsProject, QgsCoordinateReferenceSystem, QgsMultiBandColorRenderer, QgsCoordinateTransform, QgsRasterProjector, QgsRasterFileWriter, QgsRasterPipe
import os

# Supply path to qgis install location
QgsApplication.setPrefixPath("C:/OSGeo4W/apps/qgis-ltr", True)

# Create a reference to the QgsApplication. Setting the second argument to False disables the GUI.
qgs = QgsApplication([], False)

# Load providers
qgs.initQgis()

# Create a new QGIS project
project = QgsProject.instance()

# Define the directory where GeoTIFF files are stored
directory = 'C:/Users/hughr/OneDrive/Documents/RMI/GEE/test_images'

# Add a basemap
urlWithParams = 'type=xyz&url=https://tile.openstreetmap.org/%7Bz%7D/%7Bx%7D/%7By%7D.png&zmax=19&zmin=0&crs=EPSG3857'
basemap = QgsRasterLayer(urlWithParams, 'Basemap', 'wms')

if not basemap.isValid():
    print('Basemap failed to load.')
else:
    project.addMapLayer(basemap)

# Ensure reprojections subfolder exists
reproj_folder = os.path.join(directory, "reprojections")
os.makedirs(reproj_folder, exist_ok=True)

# Red, green, and blue bands
red_band = 16
green_band = 17
blue_band = 18    

# Loop over all the files in the directory
for filename in os.listdir(directory):
    # Check if the file is a GeoTIFF
    if filename.endswith('.tif') or filename.endswith('.tiff'):
        # Define the full file path
        filepath = os.path.join(directory, filename)
        # Create a new raster layer
        layer = QgsRasterLayer(filepath, filename)

        # Set RGB bands
        renderer = QgsMultiBandColorRenderer(layer.dataProvider(), red_band, green_band, blue_band)
        layer.setRenderer(renderer)
        
        # If the layer's CRS is not EPSG:3857, reproject it
        if layer.crs().authid() != 'EPSG:3857':
            # Define the target CRS
            target_crs = QgsCoordinateReferenceSystem('EPSG:3857')
            # Define a coordinate transform
            transform = QgsCoordinateTransform(layer.crs(), target_crs, QgsProject.instance().transformContext())

            # Reproject the raster
            pipe = QgsRasterPipe()
            provider = layer.dataProvider()
            
            try:
                #pipe.set(layer.dataProvider())
                pipe.set(provider.clone())
                pipe.set(renderer.clone())
            except:
                print("Could not set data provider")
                continue

            projector = QgsRasterProjector()
            projector.setCrs(layer.crs(), target_crs, QgsProject.instance().transformContext())

            try:
                pipe.insert(2, projector)
            except:
                print("Could not insert projector into pipe")
                continue

            # Create a new raster file writer with the reprojected file path
            reproj_filename = filename.split(".")[0] + '_reprojected.tif'
            output_file = os.path.join(reproj_folder, reproj_filename)
            file_writer = QgsRasterFileWriter(output_file)

            # Set compression options
            #file_writer.setOptions(['COMPRESS=LZW'])

            # Write the reprojected raster to the new file
            file_writer.writeRaster(pipe, layer.width(), layer.height(), transform.transform(layer.extent()), target_crs)

            # Load the reprojected layer
            layer = QgsRasterLayer(output_file, filename + ' Reprojected')

        # Add the layer to the project
        if not layer.isValid():
            print('Layer failed to load: ' + filepath)
        else:
            project.addMapLayer(layer)


# Save the project to a .qgs file
project.write('C:/Users/hughr/OneDrive/Documents/RMI/GEE/test_gee.qgs')

# Close the QgsApplication. This will remove any temporary files and cleanup 
qgs.exitQgis()

C:\Users\hughr\AppData\Local\Temp\ipykernel_38040\3461910827.py:87: DeprecationWarning: QgsRasterFileWriter.writeRaster() is deprecated
  file_writer.writeRaster(pipe, layer.width(), layer.height(), transform.transform(layer.extent()), target_crs)
